# Анализ решений по апелляциям на постановления ГИБДД

## п. 2. Предобработка решений
Цель данного этапа:   
Предобработать решения - выделение метаданных и признаков: дата вынесения постановления, решения, город, суд, судья, инстанция, наличие представителя привлекаемого лица в суде (юрист/адвокат), номер статьи, доказательная база (наличие фото/видео материалов, свидетелей, понятых), тип решения (удовлетворить/частично/отказать...), возможно что-то ещё.

## 1. Creating df of resolusions filtrated be parameters

Существующая база решений всех судом РФ за 2010...2013 годы нахиодится в папке data/sou/ и имеет следующйю структуру: data/sou/court_name/year/resolution_n.xml

В данном проекте будут рассматриваться только дела одновременно соответствующие всем следующим критериям:
1. Вид судопроизводства: административное (Дела об админитсративных правонарушениях)
2. Этап: Апелляция
3. Статьи: главы 12 КоАП РФ (правонарушения в сфере ПДД)
4. Вид документа: решение
5. Первая инстанция: ГИБДД (инспектор ДПС)
6. Суд: все районные суды г. Москвы

In [44]:
from glob import glob
from tqdm import tqdm
import os
from bs4 import BeautifulSoup
from IPython.core.display import display, HTML
import pandas as pd
import numpy as np
import re

In [2]:
#Structure
glob('../../data/sou/*')

['../../data/sou/svobodnenskij-rajonnyj-sud-amurskaya-oblast-1759',
 '../../data/sou/motovilixinskij-rajonnyj-sud-g-permi-permskij-kraj-1277',
 '../../data/sou/arkadakskij-rajonnyj-sud-saratovskaya-oblast-190',
 '../../data/sou/oktyabrskij-rajonnyj-sud-g-tomska-tomskaya-oblast-1468',
 '../../data/sou/tunkinskij-rajonnyj-sud-respublika-buryatiya-2034',
 '../../data/sou/zarechenskij-gorodskoj-sud-penzenskaya-oblast-654',
 '../../data/sou/ulyanovskij-rajonnyj-sud-ulyanovskaya-oblast-2067',
 '../../data/sou/bogorodskij-gorodskoj-sud-nizhegorodskaya-oblast-312',
 '../../data/sou/sosnovskij-rajonnyj-sud-nizhegorodskaya-oblast-1875',
 '../../data/sou/plavskij-rajonnyj-sud-tulskaya-oblast-1584',
 '../../data/sou/sakmarskij-rajonnyj-sud-orenburgskaya-oblast-1723',
 '../../data/sou/pervomajskij-rajonnyj-sud-g-krasnodara-krasnodarskij-kraj-1547',
 '../../data/sou/staropoltavskij-rajonnyj-sud-volgogradskaya-oblast-1895',
 '../../data/sou/voskresenskij-rajonnyj-sud-nizhegorodskaya-oblast-440',
 '..

In [3]:
#Creating list of all Moscow district courts
all_courts_list = glob('../../data/sou/*') #all courts
sub = 'rajonnyj-sud-gorod-moskva' #sub name of folder of Moscow district courts
msk_districts_courts_list = sorted([s for s in all_courts_list if sub in s]) # sorted list of all Moscow district courts
print('Районных судов в Москве: {}'.format(len(msk_districts_courts_list)))
msk_districts_courts_list

Районных судов в Москве: 33


['../../data/sou/babushkinskij-rajonnyj-sud-gorod-moskva-218',
 '../../data/sou/basmannyj-rajonnyj-sud-gorod-moskva-247',
 '../../data/sou/butyrskij-rajonnyj-sud-gorod-moskva-368',
 '../../data/sou/cheremushkinskij-rajonnyj-sud-gorod-moskva-2205',
 '../../data/sou/chertanovskij-rajonnyj-sud-gorod-moskva-2225',
 '../../data/sou/dorogomilovskij-rajonnyj-sud-gorod-moskva-563',
 '../../data/sou/gagarinskij-rajonnyj-sud-gorod-moskva-462',
 '../../data/sou/golovinskij-rajonnyj-sud-gorod-moskva-479',
 '../../data/sou/izmajlovskij-rajonnyj-sud-gorod-moskva-706',
 '../../data/sou/koptevskij-rajonnyj-sud-gorod-moskva-934',
 '../../data/sou/kuncevskij-rajonnyj-sud-gorod-moskva-1047',
 '../../data/sou/kuzminskij-rajonnyj-sud-gorod-moskva-1030',
 '../../data/sou/lefortovskij-rajonnyj-sud-gorod-moskva-1148',
 '../../data/sou/lyublinskij-rajonnyj-sud-gorod-moskva-1178',
 '../../data/sou/meshhanskij-rajonnyj-sud-gorod-moskva-1232',
 '../../data/sou/nagatinskij-rajonnyj-sud-gorod-moskva-1298',
 '../../

In [87]:
%%time
#creating df of all_msk-districts_courts

cols = ['region', 'court', 'judge', 'vidpr', 'etapd', 'category', 'result', 'date', 'vid_dokumenta', 'file']
df_msk_districts = pd.DataFrame(columns=cols)


for court in msk_districts_courts_list: 
    print(court)
    path = court
    years_list = glob(os.path.join(path, '*'))
    for year in years_list:
        path = year
        files = glob(os.path.join(path, '*'))
        for file in tqdm(files):
            with open(file, 'r', encoding='utf-8') as fd:
                doc = fd.read()
            parsed_html = BeautifulSoup(doc)
            df_msk_districts = df_msk_districts.append(
                {
                    'region': parsed_html.region.text,
                    'court': parsed_html.court.text,
                    'judge': parsed_html.judge.text,
                    'vidpr': parsed_html.vidpr.text,
                    'etapd': parsed_html.etapd.text,
                    'category': parsed_html.category.text,
                    'result': parsed_html.result.text,
                    'date': parsed_html.date.text,
                    'vid_dokumenta': parsed_html.vid_dokumenta.text,
                    'file': file
                }, 
                ignore_index=True
            )

  1%|          | 15/1964 [00:00<00:13, 149.31it/s]

../../data/sou/babushkinskij-rajonnyj-sud-gorod-moskva-218


  0%|          | 20/4618 [00:00<00:23, 191.78it/s]

../../data/sou/basmannyj-rajonnyj-sud-gorod-moskva-247


  1%|          | 14/2707 [00:00<00:19, 136.66it/s]

...

../../data/sou/zelenogradskij-rajonnyj-sud-gorod-moskva-668


  0%|          | 3/1178 [00:00<00:58, 20.24it/s]

../../data/sou/zyuzinskij-rajonnyj-sud-gorod-moskva-691


100%|██████████| 1266/1266 [01:04<00:00, 19.77it/s]

CPU times: user 1h 20min 9s, sys: 39.1 s, total: 1h 20min 48s
Wall time: 1h 20min 15s


In [550]:
#save to csv
df_msk_districts.to_csv('df_msk_districts.csv', index=False)

In [45]:
#load from csv back
df_msk_districts = pd.read_csv('df_msk_districts.csv')
df_msk_districts

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file
0,Москва,Бабушкинский районный суд (Город Москва),Скулина Юлия Андреевна,Гражданское,Первая инстанция,Иски о взыскании сумм по договору займа,Иск (заявление) удовлетворен (в том числе част...,30.03.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
1,Москва,Бабушкинский районный суд (Город Москва),Романцова Татьяна Алексеевна,Гражданское,Первая инстанция,Прочие исковые дела,Иск (заявление) удовлетворен (в том числе част...,09.02.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
2,Москва,Бабушкинский районный суд (Город Москва),Романцова Татьяна Алексеевна,Гражданское,Первая инстанция,"Другие споры, связанные с землепользованием",Иск (заявление) удовлетворен (в том числе част...,20.01.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
3,Москва,Бабушкинский районный суд (Город Москва),Рябова Елена Вячеславовна,Гражданское,Первая инстанция,Иски о возмещении ущерба от ДТП,Иск (заявление) удовлетворен (в том числе част...,13.07.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4,Москва,Бабушкинский районный суд (Город Москва),Мартыненко Антон Анатольевич,Уголовное,Первая инстанция,162 ч.2,Обвинительный приговор,18.05.2012,приговор,../../data/sou/babushkinskij-rajonnyj-sud-goro...
5,Москва,Бабушкинский районный суд (Город Москва),Никитин Владимир Федорович,Уголовное,Первая инстанция,228.1 ч.1,Обвинительный приговор,21.08.2012,приговор,../../data/sou/babushkinskij-rajonnyj-sud-goro...
6,Москва,Бабушкинский районный суд (Город Москва),Рябова Елена Вячеславовна,Гражданское,Первая инстанция,Трудовые споры (независимо от форм собственнос...,Отказано,10.02.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
7,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Гражданское,Первая инстанция,"О защите чести, достоинства, деловой репутации...",Иск (заявление) удовлетворен (в том числе част...,27.07.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
8,Москва,Бабушкинский районный суд (Город Москва),Чернышева Татьяна Викторовна,Гражданское,Первая инстанция,Иски о взыскании платы за жилую площадь и комм...,Иск (заявление) удовлетворен (в том числе част...,10.02.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
9,Москва,Бабушкинский районный суд (Город Москва),Романцова Татьяна Алексеевна,Гражданское,Первая инстанция,Прочие исковые дела,Иск (заявление) удовлетворен (в том числе част...,13.07.2012,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...


In [46]:
# create sub df of court resolutions on appeal by administrative cases
vidpr = 'Административное'
etapd = 'Апелляция'
vid_dokumenta = 'решение'
category = '12.'

df_12koap=df_msk_districts[(df_msk_districts.vidpr==vidpr)&(df_msk_districts.etapd==etapd)& \
                 (df_msk_districts.vid_dokumenta==vid_dokumenta)&
                 (df_msk_districts['category'].apply(lambda x: True if str(x).find('12.')>=0 else False) == True)]

df_12koap

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file
3125,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.27 ч.2,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
3291,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4446,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.26 ч.1,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4521,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.26 ч.1,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
4728,Москва,Бабушкинский районный суд (Город Москва),Иваненко Юлия Сергеевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.08.2011,решение,../../data/sou/babushkinskij-rajonnyj-sud-goro...
5292,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.8 ч.1,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...
5327,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,18.01.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...
5396,Москва,Басманный районный суд (Город Москва),Дударь Наталия Николаевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,13.09.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...
5400,Москва,Басманный районный суд (Город Москва),Дударь Наталия Николаевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,13.09.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...
5495,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,23.10.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...


## 2. Preprocessing

### 2.1 Reveiw some statistics

In [58]:
# get statictics (absolute numbers)
def df_statistics(df):
    for col in df.columns:
        print(col)
        print('Absolute values: \n')
        print(df[col].value_counts())
        print('\n')
        print('Percentages: \n')
        print(df[col].value_counts()/len(df)*100)
        print('-'*40)

In [124]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

### 2.2 Editing categories' values

Видим, что в качестве значения признака category есть такие значения как '12.26 ч.1; 12.26' или '12.26; 12.26 ч.1', которые являются другим видом записи значения признака '12.26 ч.1'.
Приведём все такие записи значений в единый вид.

In [125]:
# edit categories like '12.xx ч.y; 12.xx or 12.xx; 12.xx ч.y' into '12.xx ч.y'
def edit_category(category):
    import re
    if type(category) != str: stat_part = category
    elif len(re.findall(r'12\.\d+(?:\.\d)? ч.\d+; 12\.\d+(?:\.\d)?', category)) == 1:
        stat_part = re.findall(r'(12\.\d+(?:\.\d)? ч.\d+); 12\.\d+(?:\.\d)?', category)[0]
    elif len(re.findall(r'12\.\d+(?:\.\d)?; 12\.\d+ ч.\d+(?:\.\d)?', category)) == 1:
        stat_part = re.findall(r'12\.\d+(?:\.\d)?; (12\.\d+ ч.\d+(?:\.\d)?)', category)[0]
    else: stat_part = category
    return stat_part

In [89]:
df_12koap.loc[:,'category'] = df_12koap['category'].apply(edit_category)

In [126]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

Теперь все значения приобразованы к единому виду. Такие значения как '12.26 ч.1; 12.26' или '12.26; 12.26 ч.1' исчезли.

### 2.3 Editing results' values

Значение признака result:  
'Отменено c возвращением на новое рассмотрение' то же самое что и 'Отменено возвращением на новое рассмотрение',  
'Производство по делу прекращено' то же самое что и 'Отменено с прекращением производства'.  
Изменим все одинаковые по сути значения на те, которые используются чаще.

In [216]:
# edit the same decisions into single type
df_12koap.replace('Отменено с возвращением на новое рассмотрение', \
                  'Отменено возвращением на новое рассмотрение', inplace=True)
df_12koap.replace('Производство по делу прекращено', \
                  'Отменено с прекращением производства', inplace=True)

In [217]:
df_statistics(df_12koap)

region
Absolute values: 

Москва    3317
Name: region, dtype: int64


Percentages: 

Москва    100.0
Name: region, dtype: float64
----------------------------------------
court
Absolute values: 

Черемушкинский районный суд (Город Москва)    544
Зеленоградский районный суд (Город Москва)    458
Кузьминский районный суд (Город Москва)       414
Симоновский районный суд (Город Москва)       248
Хорошевский районный суд (Город Москва)       218
Бутырский районный суд (Город Москва)         190
Басманный районный суд (Город Москва)         181
Солнцевский районный суд (Город Москва)       133
Тимирязевский районный суд (Город Москва)     132
Лефортовский районный суд (Город Москва)      123
Перовский районный суд (Город Москва)         118
Останкинский районный суд (Город Москва)      104
Чертановский районный суд (Город Москва)       83
Коптевский районный суд (Город Москва)         77
Измайловский районный суд (Город Москва)       60
Головинский районный суд (Город Москва)        47
Нику

### 2.4 Add feature 'first instance'

Не все постановления по статьям 12 гл. КоАП выносятся ГИБДД. Согласно ч.1 ст. 23.1 КоАП РФ есть перечень статей, по которым постановление выносит мировой судья судебного участка, однако согласно ч. 2 той же статьи есть другой перечень статей, по которым постановление может вынести как ГИБДД, так и судья в случае, если сотруднк ГИБДД передаст это дело на рассмотрение в суд. Поэтому одназначно разделить подсудность между ГИБДД и судьями только на основании статьи невозможно. Кроме того, в пследние годы постановления по 12. гл. КоАП также стали выносить сотрудники таких организаций как МАДИ и АМПП (в Москве), в других регионах возможно тоже есть аналогичные службы, которые наряду с ГИБДД и судьями наделены полномочиями выносить постановления по 12 гл. КоАП РФ.   

Для определения первой инстанции: 
- 1) сначла проверяем докумнет по номеру статьи, если по данной статье решения однозначно выносятся только мировым судьёй судебного участка - то присваиваем этому документу в качестве 1-й инстанции - "Мировой суд", 
- 2) если же статья не входит в список статей, которые рассматриваются исключительно судьями, то 1-я инстанция определяется с помощью поиска ключевых слов и словосочетаний в тексте решений.

#### 2.4.1 Creating list of stats for mirovoy_sud only

В данной части сформируем список статей по которым постановление принимает не инспектор ГИБДД, а мировой судья.
Данный перечень содержится в тексте главы 23 КоАП РФ. Для формирования списка статей в виде, используемом в судебных актах воспользуемся регулярными выражениями.

In [36]:
# convert text with comma separaterd stats to list of standard veiw stats
def text_to_stats(text, gl='12'):
    #separate text into lines like 'parts...of stat...'
    import re
    mir_sud_cats = re.split(r'(?<='+gl+r'.\d), |(?<='+gl+r'.\d\d), |(?<='+gl+r'.\d\d.\d), |(?<=[)]), ', text)
    
    #convert stats to standard form
    stat_parts = []
    for category in mir_sud_cats:
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) == 1:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            part = re.findall(r' (\d(?:\.\d)?)\b', category)[0]
            stat_part = stat+' ч.'+part
            stat_parts.append(stat_part)
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) == 0:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            stat_part = stat
            stat_parts.append(stat_part)
        if len(re.findall(r' (\d(?:\.\d)?)\b', category)) > 1:
            stat = re.findall(r''+gl+r'\.\d+(?:\.\d)?', category)[0]
            parts = list(re.findall(r' (\d(?:\.\d)?)\b', category))
            if len(re.findall(r'\d - \d', category)) >=1:
                start = int(re.findall(r'\d - \d', category)[0][0])
                end = int(re.findall(r'\d - \d', category)[0][-1])
                parts.extend(str(i) for i in range(start, end+1))
                parts = sorted(set(parts))
            for part in parts:
                stat_part = stat+' ч.'+part
                stat_parts.append(stat_part)
    return stat_parts

Подсудность дел об административных правонарушениях

Мировой судья рассматривает дела об административных правонарушениях предусмотренные КоАП РФ в пределах компетенции установленной главой 23 КоАП РФ.

ч.1 ст. 23.1 КоАП РФ: (только статьи по Главе 12 КоАП РФ - нарущения в сфере ПДД)

... частью 4 статьи 12.2, частью 2.1 статьи 12.3, частями 1, 2, частью 3 (в случаях незаконного нанесения цветографической схемы легкового такси) статьи 12.4, частями 3, 4 - 7 статьи 12.5, частью 2 статьи 12.7, статьей 12.8, частью 7 статьи 12.9 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи), частью 3 статьи 12.10, частью 5 статьи 12.15 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), частью 3.1 статьи 12.16 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), статьями 12.24, 12.26, частями 2 и 3 статьи 12.27, статьями 12.34, 12.35, ...

In [37]:
mir_sud_text = 'частью 4 статьи 12.2, частью 2.1 статьи 12.3, частями 1, 2, частью 3 (в случаях незаконного нанесения цветографической схемы легкового такси) статьи 12.4, частями 3, 4 - 7 статьи 12.5, частью 2 статьи 12.7, статьей 12.8, частью 7 статьи 12.9 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи), частью 3 статьи 12.10, частью 5 статьи 12.15 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), частью 3.1 статьи 12.16 (за исключением случаев фиксации административного правонарушения работающими в автоматическом режиме специальными техническими средствами, имеющими функции фото- и киносъемки, видеозаписи, или средствами фото- и киносъемки, видеозаписи), статьями 12.24, 12.26, частями 2 и 3 статьи 12.27, статьями 12.34, 12.35'

Постановления по данным частям могут выноситься сотрудниками ГИБДД, поэтому их удаляем:
<br>
частью 3 статьи 12.4 - ГИБДД
<br>
частью 7 статьи 12.9 (повторное превышение скорости) - ЦАФАП ГИБДД
<br>
частью 5 статьи 12.15 (повторный выезд на стречную полосу) - ГИБДД ЦАФАП
<br>
частью 3.1 статьи 12.16 (повторное движение во встречном направлении по односторонней дороге) - ГИБДД ЦАФАП

In [38]:
#creating list of gibdd_stats
text_gibdd_stats = 'частью 3 статьи 12.4, частью 7 статьи 12.9, частью 5 статьи 12.15, частью 3.1 статьи 12.16'
gibdd_stats = text_to_stats(text_gibdd_stats)
gibdd_stats

['12.4 ч.3', '12.9 ч.7', '12.15 ч.5', '12.16 ч.3.1']

In [39]:
#creating list of mir_sud_stats withot gibdd_stats
mir_sud_stat_parts = []
mir_sud_stat_parts = text_to_stats(mir_sud_text)
#del_gibdd_stats
mir_sud_stat_parts = set(mir_sud_stat_parts)
mir_sud_stat_parts.difference_update(set(gibdd_stats))
mir_sud_stat_parts

{'12.10 ч.3',
 '12.2 ч.4',
 '12.24',
 '12.26',
 '12.27 ч.2',
 '12.27 ч.3',
 '12.3 ч.2.1',
 '12.34',
 '12.35',
 '12.4 ч.1',
 '12.4 ч.2',
 '12.5 ч.3',
 '12.5 ч.4',
 '12.5 ч.5',
 '12.5 ч.6',
 '12.5 ч.7',
 '12.7 ч.2',
 '12.8'}

#### 2.4.2 Creating functions for text analisys

In [49]:
#check if resolutions of mirovoy sud
def is_mir_sud(text):
    import re
    if len(re.findall(r'[Сс]удебн\w+ [Уу]част\w+ |[Мм]иров\w+ [Сс]удь\w+', text)) > 0: return True
    else: return False

In [50]:
#check if resolutions of GIBDD
def is_gibdd(text):
    import re
    if len(re.findall(r'ГИБДД|ДПС|МВД|[Аа]втоматическ\w+', text)) > 0: return True
    elif (len(re.findall(r'[Ии]нспектор', text))>0)&(len(re.findall(r'12.', text))>0): return True
    else: return False

#### 2.4.3 Creating first_instance feature

In [60]:
def first_instance(file, mir_sud_stats = mir_sud_stat_parts):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    if BeautifulSoup(doc).category.text in mir_sud_stats: return 'Мировой суд'
    else:  
        text = BeautifulSoup(doc).body.text
        if is_mir_sud(text): return 'Мировой суд'
        elif is_gibdd(text): return 'ГИБДД'
        else: return 'Не определено'

In [64]:
df_12koap.loc[:,'first_instance'] = df_12koap['file'].apply(first_instance)
df_12koap[df_12koap['first_instance'] == 'Не определено']

,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance
29495,Москва,Чертановский районный суд (Город Москва),Маркина Надежда Николаевна,Административное,Апелляция,12.8 ч.1,Оставлено без изменения,29.11.2010,решение,../../data/sou/chertanovskij-rajonnyj-sud-goro...,Не определено
30700,Москва,Чертановский районный суд (Город Москва),Тюркина Галина Михайловна,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,25.03.2011,решение,../../data/sou/chertanovskij-rajonnyj-sud-goro...,Не определено
66156,Москва,Кузьминский районный суд (Город Москва),Езерская Жанна Анатольевна,Административное,Апелляция,12.13 ч.2,Оставлено без изменения,17.09.2012,решение,../../data/sou/kuzminskij-rajonnyj-sud-gorod-m...,Не определено
113155,Москва,Останкинский районный суд (Город Москва),Ложкомоев Александр Юрьевич,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,08.11.2011,решение,../../data/sou/ostankinskij-rajonnyj-sud-gorod...,Не определено


Посмотрим кто выносил Постановления из тех, которые выносили не мировые судьи

In [159]:
def get_files_list(df):
    print('Всего документов: {}'.format(len(df)))
    return df.file.to_list()

In [161]:
#get text of sud act
def get_sud_text(files_list, doc_number):
    file = files_list[doc_number]
    with open(file, 'r', encoding='utf-8') as fd:
        doc = fd.read()
    parsed_html = BeautifulSoup(doc)
    return parsed_html.body.text

In [203]:
df = df_12koap[df_12koap['first_instance'] == 'Не определено']
no_mir_suf_list = get_files_list(df)

Всего документов: 4


В оставшихся 4 документах, где не определена 1-я инстанция: либо в метаданных неверно указана статья КоАП и в действительности не относится к гл. 12 либо из текста не очевидна первая инстанция

In [207]:
doc_number = 0

display(HTML(get_sud_text(no_mir_suf_list, doc_number)))

### 2.5 Creating sub df with only GIBDD first instance

In [210]:
df_gibdd = df_12koap[df_12koap.first_instance == 'ГИБДД']
print('Всего документов: {}'.format(len(df_gibdd)))
df_gibdd

Всего документов: 331


,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance
6114,Москва,Басманный районный суд (Город Москва),Карпов Артур Геннадьевич,Административное,Апелляция,12.16 ч.1,Оставлено без изменения,06.11.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
7447,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
8919,Москва,Басманный районный суд (Город Москва),Дударь Н. Н.,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,05.07.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9005,Москва,Басманный районный суд (Город Москва),Дударь Наталия Николаевна,Административное,Апелляция,12.14 ч.3,Оставлено без изменения,13.09.2012,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9784,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.16 ч.5,Оставлено без изменения,07.03.2013,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9796,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.16 ч.5,Оставлено без изменения,07.03.2013,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
9804,Москва,Басманный районный суд (Город Москва),Скуридина Ирина Анатольевна,Административное,Апелляция,12.16 ч.1,Оставлено без изменения,06.03.2013,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
10476,Москва,Басманный районный суд (Город Москва),Карпов А. Г.,Административное,Апелляция,12.13 ч.2,Оставлено без изменения,03.11.2010,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
10661,Москва,Басманный районный суд (Город Москва),Скуридина И. А.,Административное,Апелляция,12.19 ч.4,Оставлено без изменения,25.05.2011,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД
11171,Москва,Басманный районный суд (Город Москва),Мушникова Н. Е.,Административное,Апелляция,12.15 ч.4,Оставлено без изменения,02.06.2011,решение,../../data/sou/basmannyj-rajonnyj-sud-gorod-mo...,ГИБДД


### 2.6 Fill NaN

In [211]:
df_gibdd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 6114 to 172816
Data columns (total 11 columns):
region            331 non-null object
court             331 non-null object
judge             329 non-null object
vidpr             331 non-null object
etapd             331 non-null object
category          331 non-null object
result            311 non-null object
date              331 non-null object
vid_dokumenta     331 non-null object
file              331 non-null object
first_instance    331 non-null object
dtypes: object(11)
memory usage: 31.0+ KB


Есть пропуски по судьям и пропуски по результату. Судьи нас пока не сильно волнуют, а вот результат важен. Посмотрим на данные тексты и решим как можно заполнить пропуски по результатам

In [262]:
def get_result(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'[Пп]рекрат\w+', text)) > 0: return 'Отменено с прекращением производства'
    elif len(re.findall(r'[Оо]тказать', text)) > 0: return 'Оставлено без изменения'
    else: return 'Не определено'

In [264]:
df_gibdd.loc[(df_gibdd['result'].isna()==True)|(df_gibdd['result']== 'Не определено'), 'result'] = \
    df_gibdd.loc[(df_gibdd['result'].isna()==True), 'file'].apply(get_result)
df_gibdd[df_gibdd['result'] == 'Не определено']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,region,court,judge,vidpr,etapd,category,result,date,vid_dokumenta,file,first_instance


In [268]:
df = df_gibdd[df_gibdd['result'] == 'Не определено']
na_list = get_files_list(df)

doc_number = 0
#display(HTML(get_sud_text(na_list, doc_number)))

Всего документов: 0


## 3. Adding keywords from text to features

### 3.1 Searching and adding 'Представитель/Адвокат/Защитник'

In [ ]:
#В текстах судебных актов упоминается представтель, даже если по факту представителя привлекаемого лица нет, 
#проверим акты в ручную и посмотрим в каких случаях представитель имеется, а в каких случаях нет

#Наличие представителя:
#Представитель Шутовой Е.В. по доверенности
#В судебном заседании представитель Горбушин жалобу поддержал.
#с участием ФИО5, представителя ФИО2,
#ФИО5 и его представитель ФИО6 доводы жалобы поддержали
#с участием заявителя ФИО3 и его представителя ФИО7
#Заявитель ФИО3 и его представитель по доверенности ФИО7
#ФИО1 и ее представитель в суд явились, доводы жалобы поддержали
#ФИО4 и его представитель в суд не явились, дело просили рассмотреть в их отсутствие (л.д.64-65).
#Представитель Болдырь Н.И. просил отменить постановление
#Представитель заявителя в судебное заседание не явился, извещен.
#Представитель ЗАО « Мерседес Бенц Рус» просил отменить постановление
#Представитель заявителя в судебное заседание не явился, просил рассмотреть дело в свое отсутствие.
#Представитель ООО « Экстенал Менеджмент Груп» просил отменить постановление
#Заявитель и его представитель по доверенности ФИО3 в судебное заседание явились

#Отсутствие представителя:
#помощью защитника или представителя Героев И.Б. не воспользовался
#Заслушиваются объяснения физического лица или законного представителя юридического лица
#представитель МГУП «Мосводоканал»
#законным представителем физического лица, в том числе, несовершеннолетнего, законным представителем юридического лица, защитником и представителем.
#Представитель ЦАФАП РДД ГИБДД ГУ МВД России
#Представитель ОБ ГИБДД ... по ... ... в судебное заседание не явился, извещен надлежащим образом.
#Представитель ОБ ГИБДД ... по ЗАО ... в судебное заседание явился
#Представитель ЦАФАП ОДД ГИБДД ГУ МВД России
#законным представителем юридического лица, защитником и представителем.

In [442]:
# Проверка наличия представителя/защитника/адвоката привлекаемого лица
def get_representative_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    #представитель
    if (len(re.findall(r'[Пп]редставител\w+', text)) > 0)&(len(re.findall(r'по доверенности', text)) > 0): return True
    elif len(re.findall(r'\b(?:его|е[её]) представител\w+', text)) > 0: return True
    elif len(re.findall(r'представитель \w+ жалобу', text)) > 0: return True
    elif len(re.findall(r'представител\w+ ФИО', text)) > 0: return True
    elif (len(re.findall(r'[Пп]редставител\w+ (?:\w+)+ просил отменить постановление', text)) > 0) : return True
    #&(len(re.findall(r'просил отменить постановление', text)) > 0): return True
    elif len(re.findall(r'[Пп]редставитель заявител\w+', text)) > 0: return True
    #адвокат
    elif len(re.findall(r'(?<!отсутствии) [Аа]двокат\w+', text)) > 0: return True
    #защитник
    elif (len(re.findall(r'[Зз]ащитник\w+', text)) > 0)&\
        (len(re.findall(r'[Зз]ащитник\w+ (?:и|или) [Пп]редставител\w+', text)) == 0)&\
        (len(re.findall(r'[Пп]омощ\w+ [Зз]ащитник\w+', text)) == 0): return True
    else: return False

In [462]:
text = 'Представитель ОБ дпс просил отменить постановление'
re.findall(r'[Пп]редставител\w+ (?:\w+){1,3} просил отменить постановление', text)

[]

In [443]:
df_gibdd.loc[:,'representative'] = df_gibdd['file'].apply(get_representative_existence)
df_gibdd.representative.value_counts()

False    250
True      81
Name: representative, dtype: int64

In [447]:
df = df_gibdd[df_gibdd['representative'] == True]
na_list = get_files_list(df)

doc_number = 50
display(HTML(get_sud_text(na_list, doc_number)))

#выборочная проверка документов не обнаружила ошибок определния наличия представителя/адвоката/защитника привлекаемого лица

Всего документов: 81


### 3.2 Searching and adding 'Фото'

In [270]:
def get_representative_existence(file):
    with open(file, 'r', encoding='utf-8') as fd: doc = fd.read()
    text = BeautifulSoup(doc).body.text
    if len(re.findall(r'[Пп]редставител\w+', text)) > 0: return True
    else: return False

In [273]:
df_gibdd.loc[:,'representative'] = df_gibdd['file'].apply(get_representative_existence)
df_gibdd.representative.value_counts()

False    271
True      60
Name: representative, dtype: int64